In [51]:
import pandas as pd

# ============================
# 1. Cargar datasets
# ============================
df1 = pd.read_csv("C:/Users/juana/Desktop/liquideuda_project/datasets_clean/merged_registradores.csv")
df2 = pd.read_csv("C:/Users/juana/Desktop/liquideuda_project/originales/INE_PARO_EDAD_SEXO_CA.csv", sep=";")

# ============================
# 2. Explorar las CCAA en df2 (paro)
# ============================
print("Comunidades Autónomas en df2 (Paro):")
print(df2["Comunidades y Ciudades Autónomas"].unique())




Comunidades Autónomas en df2 (Paro):
['Total Nacional' '01 Andalucía' '02 Aragón' '03 Asturias, Principado de'
 '04 Balears, Illes' '05 Canarias' '06 Cantabria' '07 Castilla y León'
 '08 Castilla - La Mancha' '09 Cataluña' '10 Comunitat Valenciana'
 '11 Extremadura' '12 Galicia' '13 Madrid, Comunidad de'
 '14 Murcia, Región de' '15 Navarra, Comunidad Foral de' '16 País Vasco'
 '17 Rioja, La' '18 Ceuta' '19 Melilla']


In [52]:
df2.columns

Index(['Sexo', 'Comunidades y Ciudades Autónomas', 'Edad', 'Periodo', 'Total'], dtype='object')

In [53]:
# ============================
# 1. Diccionarios de renombrado
# ============================

# Normalizar valores de Sexo
sexo_map = {
    "Ambos sexos": "Todos",
    "Hombres": "Hombres",
    "Mujeres": "Mujeres"
}

# Normalizar nombres de Comunidades Autónomas
ccaa_map = {
    "Total Nacional": "España",
    "01 Andalucía": "Andalucia",
    "02 Aragón": "Aragon",
    "03 Asturias, Principado de": "Asturias",
    "04 Balears, Illes": "Baleares",
    "05 Canarias": "Canarias",
    "06 Cantabria": "Cantabria",
    "07 Castilla y León": "Castilla y Leon",
    "08 Castilla - La Mancha": "Castilla La Mancha",
    "09 Cataluña": "Cataluña",
    "10 Comunitat Valenciana": "Valencia",
    "11 Extremadura": "Extremadura",
    "12 Galicia": "Galicia",
    "13 Madrid, Comunidad de": "Madrid",
    "14 Murcia, Región de": "Murcia",
    "15 Navarra, Comunidad Foral de": "Navarra",
    "16 País Vasco": "Pais Vasco",
    "17 Rioja, La": "La Rioja",
    "18 Ceuta": "Ceuta",
    "19 Melilla": "Melilla"
}

# Normalizar valores de Edad
edad_map = {
    "Total": "Todos",
    "Menores de 25 años": "Menores25",
    "De 25 a 54 años": "25a54",
    "55 y más años": "55ymas"
}

# ============================
# 2. Aplicar cambios
# ============================
df2["Sexo"] = df2["Sexo"].replace(sexo_map)
df2["Comunidades y Ciudades Autónomas"] = df2["Comunidades y Ciudades Autónomas"].replace(ccaa_map)
df2["Edad"] = df2["Edad"].replace(edad_map)

# ============================
# 3. Validar
# ============================
print("Valores únicos después del renombrado:")
print("Sexo:", df2["Sexo"].unique())
print("CCAA:", df2["Comunidades y Ciudades Autónomas"].unique())
print("Edad:", df2["Edad"].unique())


Valores únicos después del renombrado:
Sexo: ['Todos' 'Hombres' 'Mujeres']
CCAA: ['España' 'Andalucia' 'Aragon' 'Asturias' 'Baleares' 'Canarias'
 'Cantabria' 'Castilla y Leon' 'Castilla La Mancha' 'Cataluña' 'Valencia'
 'Extremadura' 'Galicia' 'Madrid' 'Murcia' 'Navarra' 'Pais Vasco'
 'La Rioja' 'Ceuta' 'Melilla']
Edad: ['Todos' 'Menores25' '25 y más años' 'De 16 a 19 años' 'De 20 a 24 años'
 '25a54' 'De 55 y más años']


In [54]:
# Asegurar que sea string primero
df2["Total"] = df2["Total"].astype(str)

# Reemplazar comas decimales por puntos
df2["Total"] = df2["Total"].str.replace(",", ".", regex=False)

# Convertir a float
df2["Total"] = pd.to_numeric(df2["Total"], errors="coerce")

# Verificar
print(df2["Total"].describe())
print(df2["Total"].head(10))


count    39330.000000
mean        24.654217
std         18.585995
min          0.000000
25%         11.010000
50%         18.850000
75%         32.760000
max        100.000000
Name: Total, dtype: float64
0    10.29
1    11.36
2    10.61
3    11.21
4    11.27
5    12.29
6    11.80
7    11.89
8    11.67
9    13.38
Name: Total, dtype: float64


In [55]:
# Renombrar columnas de df2
df2 = df2.rename(columns={
    "Sexo": "sexo",
    "Comunidades y Ciudades Autónomas": "comunidad_autonoma",
    "Edad": "edad",
    "Periodo": "periodo",
    "Total": "tasa_paro"
})

print(df2.columns)

Index(['sexo', 'comunidad_autonoma', 'edad', 'periodo', 'tasa_paro'], dtype='object')


In [56]:
# ============================
# 3. Crear mapeo provincia → CCAA en df1
# ============================
# Diccionario provincia → Comunidad Autónoma (normalizada)
provincia_to_ccaa = {
    # Andalucía
    "Almería": "Andalucia",
    "Cádiz": "Andalucia",
    "Córdoba": "Andalucia",
    "Granada": "Andalucia",
    "Huelva": "Andalucia",
    "Jaén": "Andalucia",
    "Málaga": "Andalucia",
    "Sevilla": "Andalucia",

    # Aragón
    "Huesca": "Aragon",
    "Teruel": "Aragon",
    "Zaragoza": "Aragon",
    "Aragón": "Aragon",

    # Asturias
    "Asturias": "Asturias",

    # Baleares
    "Balears, Illes": "Baleares",

    # Canarias
    "Palmas, Las": "Canarias",
    "Santa Cruz de Tenerife": "Canarias",
    "Canarias": "Canarias",

    # Cantabria
    "Cantabria": "Cantabria",

    # Castilla y León
    "Ávila": "Castilla y Leon",
    "Burgos": "Castilla y Leon",
    "León": "Castilla y Leon",
    "Palencia": "Castilla y Leon",
    "Salamanca": "Castilla y Leon",
    "Segovia": "Castilla y Leon",
    "Soria": "Castilla y Leon",
    "Valladolid": "Castilla y Leon",
    "Zamora": "Castilla y Leon",

    # Castilla-La Mancha
    "Albacete": "Castilla La Mancha",
    "Ciudad Real": "Castilla La Mancha",
    "Cuenca": "Castilla La Mancha",
    "Guadalajara": "Castilla La Mancha",
    "Toledo": "Castilla La Mancha",

    # Cataluña
    "Barcelona": "Cataluña",
    "Girona": "Cataluña",
    "Lleida": "Cataluña",
    "Tarragona": "Cataluña",

    # Comunitat Valenciana
    "Alicante/Alacant": "Valencia",
    "Castellón/Castelló": "Valencia",
    "Valencia/València": "Valencia",
    "Valencia": "Valencia",

    # Extremadura
    "Badajoz": "Extremadura",
    "Cáceres": "Extremadura",

    # Galicia
    "Coruña, A": "Galicia",
    "Lugo": "Galicia",
    "Ourense": "Galicia",
    "Pontevedra": "Galicia",

    # Madrid
    "Madrid": "Madrid",

    # Murcia
    "Murcia": "Murcia",

    # Navarra
    "Navarra": "Navarra",

    # País Vasco
    "Araba/Álava": "Pais Vasco",
    "Bizkaia": "Pais Vasco",
    "Gipuzkoa": "Pais Vasco",

    # La Rioja
    "Rioja, La": "La Rioja",

    # Ceuta y Melilla
    "Ceuta": "Ceuta",
    "Melilla": "Melilla"
}

# Crear nueva columna en dataset 1 con la comunidad correspondiente
import pandas as pd

df1 = pd.read_csv("C:/Users/juana/Desktop/liquideuda_project/datasets_clean/merged_registradores.csv")

df1["comunidad_autonoma"] = df1["comunidad_provincia"].map(provincia_to_ccaa)

# Validar
print(df1[["comunidad_provincia", "comunidad_autonoma"]].drop_duplicates().head(20))


        comunidad_provincia comunidad_autonoma
0                   Almería          Andalucia
18                    Cádiz          Andalucia
36                  Córdoba          Andalucia
54                  Granada          Andalucia
72                   Huelva          Andalucia
90                     Jaén          Andalucia
108                  Málaga          Andalucia
126                 Sevilla          Andalucia
144                  Huesca             Aragon
162                  Teruel             Aragon
180                Zaragoza             Aragon
198                Asturias           Asturias
216          Balears, Illes           Baleares
234  Santa Cruz de Tenerife           Canarias
252                Canarias           Canarias
270               Cantabria          Cantabria
288                   Ávila    Castilla y Leon
306                  Burgos    Castilla y Leon
324                    León    Castilla y Leon
342                Palencia    Castilla y Leon


In [57]:
print("CCAA:", df1["comunidad_autonoma"].unique())

CCAA: ['Andalucia' 'Aragon' 'Asturias' 'Baleares' 'Canarias' 'Cantabria'
 'Castilla y Leon' 'Valencia' 'Castilla La Mancha' 'Cataluña'
 'Extremadura' 'Galicia' 'Madrid' 'Murcia' 'Navarra' 'Pais Vasco' 'Ceuta'
 'Melilla']


In [58]:
# Filtrar solo filas donde la comunidad_autonoma es NaN
nan_provincias = (
    df1[df1["comunidad_autonoma"].isna()]
    .groupby("comunidad_provincia")
    .size()
    .reset_index(name="n_nulos")
    .sort_values("n_nulos", ascending=False)
)

print("Provincias con comunidad_autonoma = NaN:")
print(nan_provincias)



Provincias con comunidad_autonoma = NaN:
Empty DataFrame
Columns: [comunidad_provincia, n_nulos]
Index: []


CAMBIAR PERIODO A DOS CAMPOS

In [59]:
# Primero, crear las columnas 'año' y 'trimestre' desde 'periodo'
df2["año"] = df2["periodo"].str[:4].astype(int)
df2["trimestre"] = df2["periodo"].str[-1].astype(int)

# Ahora filtramos solo desde 2021 en adelante
df2 = df2[df2["año"] >= 2021].reset_index(drop=True)

# Verificamos que no queden años anteriores
print(df2["año"].unique())



[2025 2024 2023 2022 2021]


INE_ ASALARIADOS X INDUSTRIAS

In [60]:
import pandas as pd
import re



# Leer el dataset original
df3 = pd.read_csv(
    r"C:\Users\juana\Desktop\liquideuda_project\originales\INE_ASALARIADOS_INDUSTRIAS_PROV.csv",
    sep=";",
    encoding="utf-8"
)

# Renombrar columnas para estandarizar
df3 = df3.rename(columns={
    "Provincias": "provincia",
    "Divisiones CNAE": "industria_cnae",
    "Flujos demográficos": "tipo_flujo",  # altas / bajas
    "Periodo": "periodo",
    "Total": "cantidad_flujo"
})

# Crear columna comunidad_autonoma a partir de la provincia
# --- 1. Limpiar nombres de provincia en df3 ---
# Quitamos los números iniciales y espacios extra
df3["provincia"] = df3["provincia"].apply(lambda x: re.sub(r"^\d+\s*", "", x))
provincia_to_comunidad = {
    "Almería": "Andalucia", "Cádiz": "Andalucia", "Córdoba": "Andalucia", "Granada": "Andalucia",
    "Huelva": "Andalucia", "Jaén": "Andalucia", "Málaga": "Andalucia", "Sevilla": "Andalucia",
    "Huesca": "Aragon", "Teruel": "Aragon", "Zaragoza": "Aragon",
    "Asturias": "Asturias",
    "Balears, Illes": "Baleares",
    "Palmas, Las": "Canarias", "Santa Cruz de Tenerife": "Canarias",
    "Cantabria": "Cantabria",
    "Ávila": "Castilla y Leon", "Burgos": "Castilla y Leon", "León": "Castilla y Leon",
    "Palencia": "Castilla y Leon", "Salamanca": "Castilla y Leon", "Segovia": "Castilla y Leon",
    "Soria": "Castilla y Leon", "Valladolid": "Castilla y Leon", "Zamora": "Castilla y Leon",
    "Albacete": "Castilla La Mancha", "Ciudad Real": "Castilla La Mancha", "Cuenca": "Castilla La Mancha",
    "Guadalajara": "Castilla La Mancha", "Toledo": "Castilla La Mancha",
    "Barcelona": "Cataluña", "Girona": "Cataluña", "Lleida": "Cataluña", "Tarragona": "Cataluña",
    "Alicante/Alacant": "Valencia", "Castellón/Castelló": "Valencia", "Valencia/València": "Valencia",
    "Badajoz": "Extremadura", "Cáceres": "Extremadura",
    "Coruña, A": "Galicia", "Lugo": "Galicia", "Ourense": "Galicia", "Pontevedra": "Galicia",
    "Madrid": "Madrid",
    "Murcia": "Murcia",
    "Navarra": "Navarra",
    "Araba/Álava": "Pais Vasco", "Bizkaia": "Pais Vasco", "Gipuzkoa": "Pais Vasco",
    "Rioja, La": "La Rioja",
    "Ceuta": "Ceuta",
    "Melilla": "Melilla"
}

df3["comunidad_autonoma"] = df3["provincia"].map(provincia_to_comunidad)

# ---  Excluir "Total Nacional" porque no es provincia ---
df3 = df3[df3["provincia"] != "Total Nacional"]

# Separar periodo en año y trimestre
df3["año"] = df3["periodo"].str[:4].astype(int)
df3["trimestre"] = df3["periodo"].str[-2:].str.replace("T", "").astype(int)

# Reordenar columnas para consistencia
df3 = df3[[
    "provincia", "comunidad_autonoma", "industria_cnae", "tipo_flujo", 
    "año", "trimestre", "cantidad_flujo"
]]




INFLACION INE

Index(['provincia', 'comunidad_autonoma', 'industria_cnae', 'tipo_flujo',
       'año', 'trimestre', 'cantidad_flujo'],
      dtype='object')

In [62]:
# ✅ Validación de comunidad_autonoma
print("🔍 Valores nulos en comunidad_autonoma:")
print(df3["comunidad_autonoma"].isna().sum())

print("\n🔍 Provincias sin comunidad asignada (si hay):")
print(df3[df3["comunidad_autonoma"].isna()]["provincia"].unique())

# ✅ Validación de periodo -> año y trimestre
print("\n🔍 Valores nulos en año y trimestre:")
print(df3[["año", "trimestre"]].isna().sum())

print("\n🔍 Rango de años detectados:", df3["año"].min(), "-", df3["año"].max())
print("🔍 Trimestres únicos detectados:", sorted(df3["trimestre"].unique()))

# Extra: verificar que trimestre solo contiene 1–4
if set(df3["trimestre"].unique()) <= {1, 2, 3, 4}:
    print("\n✅ Los trimestres están bien definidos (solo 1–4).")
else:
    print("\n⚠️ Atención: hay trimestres fuera del rango 1–4.")


🔍 Valores nulos en comunidad_autonoma:
0

🔍 Provincias sin comunidad asignada (si hay):
[]

🔍 Valores nulos en año y trimestre:
año          0
trimestre    0
dtype: int64

🔍 Rango de años detectados: 2020 - 2022
🔍 Trimestres únicos detectados: [1, 2, 3, 4]

✅ Los trimestres están bien definidos (solo 1–4).


In [63]:
df1.columns

Index(['id_geo___ca', 'comunidad_provincia', 's_limitadas', 's_anonimas',
       'sotras', 'personas_fisicas', 'año', 'trimestre', 't_voluntario',
       't_necesario', 't_consecutivo', 'c_ordinario', 'c_especial',
       'c_sin_masa', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k',
       'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 'na0', 'na1_2', 'na3_5',
       'na6_9', 'na10_19', 'na20_49', 'na50_99', 'na100_o_mas', 'sd_x',
       'v_menos_de_025', 'v_entre_025_y_05', 'v_entre_05_y_1', 'v_entre_1_y_2',
       'v_entre_2_y_5', 'v_entre_5_y_10', 'v_10_o_mas', 'sd_y', 'a_hasta_4',
       'a_entre_5_y_8', 'a_entre_9_y_12', 'a_entre_13_y_16', 'a_entre_17_y_19',
       'a_20_o_mas', 'comunidad_autonoma'],
      dtype='object')

In [64]:
import pandas as pd

# ---------- DF1: registradores ----------
num_cols_df1 = [c for c in df1.columns if c not in ["comunidad_provincia", "comunidad_autonoma"]]

for col in num_cols_df1:
    df1[col] = pd.to_numeric(df1[col], errors="coerce")

df1["año"] = df1["año"].astype("Int64")
df1["trimestre"] = df1["trimestre"].astype("Int64")
df1["comunidad_provincia"] = df1["comunidad_provincia"].astype("category")
df1["comunidad_autonoma"] = df1["comunidad_autonoma"].astype("category")


# ---------- DF2: paro ----------
df2["tasa_paro"] = pd.to_numeric(df2["tasa_paro"], errors="coerce")
df2["año"] = df2["año"].astype("Int64")
df2["trimestre"] = df2["trimestre"].astype("Int64")
df2["sexo"] = df2["sexo"].astype("category")
df2["comunidad_autonoma"] = df2["comunidad_autonoma"].astype("category")
df2["edad"] = df2["edad"].astype("category")
df2["periodo"] = df2["periodo"].astype("category")


# ---------- DF3: asalariados ----------
df3["cantidad_flujo"] = pd.to_numeric(df3["cantidad_flujo"], errors="coerce")
df3["año"] = df3["año"].astype("Int64")
df3["trimestre"] = df3["trimestre"].astype("Int64")
df3["provincia"] = df3["provincia"].astype("category")
df3["comunidad_autonoma"] = df3["comunidad_autonoma"].astype("category")
df3["industria_cnae"] = df3["industria_cnae"].astype("category")
df3["tipo_flujo"] = df3["tipo_flujo"].astype("category")

# ---------- Confirmación ----------
print("\nDF1 (registradores):")
print(df1.dtypes.head(15))
print("\nDF2 (paro):")
print(df2.dtypes)
print("\nDF3 (asalariados):")
print(df3.dtypes)



DF1 (registradores):
id_geo___ca               int64
comunidad_provincia    category
s_limitadas             float64
s_anonimas              float64
sotras                  float64
personas_fisicas        float64
año                       Int64
trimestre                 Int64
t_voluntario            float64
t_necesario             float64
t_consecutivo           float64
c_ordinario             float64
c_especial              float64
c_sin_masa              float64
a                       float64
dtype: object

DF2 (paro):
sexo                  category
comunidad_autonoma    category
edad                  category
periodo               category
tasa_paro              float64
año                      Int64
trimestre                Int64
dtype: object

DF3 (asalariados):
provincia             category
comunidad_autonoma    category
industria_cnae        category
tipo_flujo            category
año                      Int64
trimestre                Int64
cantidad_flujo         float64
dty

In [65]:

# Guardar df1
df1.to_csv(r"C:\Users\juana\Desktop\liquideuda_project\datasets_clean\merged_registradores.csv", 
           index=False, encoding="utf-8-sig")

# Guardar df2
df2.to_csv(r"C:\Users\juana\Desktop\liquideuda_project\datasets_clean\paro_edad_sexo_ca_ine.csv", 
           index=False, encoding="utf-8-sig")

# Guardar df3
df3.to_csv(
    r"C:\Users\juana\Desktop\liquideuda_project\datasets_clean\asalariados_industria_ine.csv",
    index=False, encoding="utf-8-sig"
)


print("Archivos guardados en datasets_clean ✅")


Archivos guardados en datasets_clean ✅
